In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### [Runnable](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html)
#### 1. Key Methods
- (a)invoke, (a)batch, (a)stream, astream_log
- built-in-optimizations for Batch, Async
- All methods accept a config argument ex. tags, tracing  

p.s. schematic information of Runnables - use property: input_schema, output_schema, config_schema

#### 2. LCEL and Composition
- LCEL: declarative way to compose runnables into chains
- chain constructed in LCEL way -> have (a)sync, batch, stream support

##### main composition primitives:
1. RunnableSequence:
- invokes runnables sequentially
- runnable -> (output) <=> (input) -> runnable
- example of using operator of RunnableSequence: **a | b | c** 
  
2. RunnableParallel
- invokes runnables cocurrently
- provide same input to each (runnables)  
runnable -> (output) <=> (input) -> runnable  
------------------------<=> (input) -> runnable  
- operate by using **dict** in the sequence ex. {'a':lambda, 'b':lambda}
- or operate by passing dict to Runnable Parallel (???)